In [61]:
# Carga paquetes
import pandas as pd
import numpy as np
import warnings
import re

warnings.filterwarnings("ignore", category=UserWarning)  # ignora los warnings

In [62]:
###########    FUNCIONES   ##########
## Interquartile Range [IQR)
#Se calculan los limites superiores e inferiores para identificación de posibles datos atipicos
# IQR = Q3 - Q1 
# LI = Q1–1.5 * IQR; LS = Q3+1.5 * IQR.
def IQR(variable):
  Iqr = np.percentile(variable,75)-np.percentile(variable,25)

  LI=np.percentile(variable,25)-(1.5*Iqr)
  LS=np.percentile(variable,75)+(1.5*Iqr)
  return(LI,LS)

In [63]:
def categorizacion (variable,outliers,reciencia):
  LI = outliers[0]
  LS = outliers[1]

  # variable tiene outliers, VarTemp no
  VarTemp = variable[(variable >= LI) & (variable <= LS)]

  ## TIPO DE INTERVALO O BREAK  
  # valores en el que se dividen los intervalos. 
  # los datos atipicos se apartan a la hora de definir los quantiles, pero se incluyen al final para el puntaje
  # break_1: <LI + min; percentiles -1 y 6; max + >LS
  breaks_1 = np.concatenate(([min(variable) - 0.001]\
    ,np.percentile(VarTemp, np.arange(0, 101, 20)[1:5]) #np.arange(start, stop, step), stop excluded so +1\
    ,[max(variable) + 0.001]))

  # break_2: quantiles con outliers
  quantiles_2 = np.percentile(variable, np.arange(0, 101, 20))
  quantiles_2[0] -= 0.001
  quantiles_2[-1] += 0.001
  breaks_2 = quantiles_2
 
  # break_3: Usandos rangos fijos y no quantiles incluyendo outliers
  breaks_3 = np.linspace(min(variable) - 0.001, max(variable) + 0.001, num=6)

  if not reciencia:
    # Usar el break sin outliers sin que quantiles repitan valores
    if not np.any(pd.Series(breaks_1).duplicated()):
        puntaje = pd.cut(variable, bins=breaks_1, labels=np.arange(1, 6)).astype(int)
        categorias = pd.cut(variable, bins=breaks_1)
    # Usar el break con outliers sin que quantiles repitan valores
    elif np.any(pd.Series(breaks_1).duplicated()) and not np.any(pd.Series(breaks_2).duplicated()):
        puntaje = pd.cut(variable, bins=breaks_2, labels=np.arange(1, 6)).astype(int)
        categorias = pd.cut(variable, bins=breaks_2)
    # Usar el break con outliers sin usar quantiles sino rangos fijos    
    else:
        puntaje = pd.cut(variable, bins=breaks_3, labels=np.arange(1, 6)).astype(int)
        categorias = pd.cut(variable, bins=breaks_3)

# restar 6 y multiplicar -1 invierte el puntaje para Recencia, es decir, si el puntaje mayor es 5 para los valores mas grandes 
# en los otros casos, en Recencia los menores son los de mayor puntaje
  else:
    if not np.any(pd.Series(breaks_1).duplicated()):
        puntaje = ((pd.cut(variable, bins=breaks_1, labels=np.arange(1, 6)).astype(int)) - 6) * (-1)
        categorias = pd.cut(variable, bins=breaks_1)
    elif np.any(pd.Series(breaks_1).duplicated()) and not np.any(pd.Series(breaks_2).duplicated()):
        puntaje = ((pd.cut(variable, bins=breaks_2, labels=np.arange(1, 6)).astype(int)) - 6) * (-1)
        categorias = pd.cut(variable, bins=breaks_2)
    else:
        puntaje = ((pd.cut(variable, bins=breaks_3, labels=np.arange(1, 6)).astype(int)) - 6) * (-1)
        categorias = pd.cut(variable, bins=breaks_3)

  return (puntaje,categorias)

In [64]:
# Función para el caso WHEN
# def case_when(x):
#     if x.isin[17,18,19,20]:
#         return 'Muy Alto'
#     elif x.isin[13,14,15,16]:
#         return 'Alto'
#     elif x.isin[8,9,10,11,12]:
#         return 'Medio'
#     else:
#         return 'Bajo'

In [75]:
# Función para el caso WHEN
def case_when(x):
    if x in [555, 554, 544, 545, 454, 455, 445]:
        return 'Leales'
    elif x in [553, 551,552, 541, 542, 533, 532, 531, 452, 451, 442, 441, 
                    431, 453, 433, 432, 423, 353, 352, 351, 342, 341, 333, 323]:
        return 'Leales en potencia'
    elif x in [512,511, 422, 421, 412, 411, 311]:
        return 'Clientes recientes'
    elif x in [525, 524, 523, 522, 521, 515, 514, 513, 425, 424, 413,414, 415, 
                    315, 314, 313]:
        return 'Prometedores'
    elif x in [535, 534, 443, 434, 343, 334, 325, 324]:
        return 'Necesitan atención'
    elif x in [331, 321, 312, 221, 213]:
        return 'A punto de dormir'
    elif x in [255, 254, 245, 244, 253, 252, 243, 242, 235, 234, 225, 224, 
                    153, 152, 145, 143, 142, 135, 134, 133, 125, 124]:
        return 'En riesgo'
    elif x in [155, 154, 144, 214,215,115, 114, 113]:
        return 'No puedes perderlos'
    elif x in [332, 322, 231, 241, 251, 233, 232, 223, 222, 132, 123, 122, 212, 211]:
        return 'Hibernando'
    elif x in [111, 112, 121, 131,141,151]:
        return 'Dormidos'
    else:
        return 'K'


In [72]:
def puntajes_RFM (tb):
  #Ejecutamos las dos funciones anteriores
  rfm_df = tb
  lim_rec = IQR(rfm_df['Recencia'])
  lim_freq = IQR(rfm_df['Frecuencia'])
  lim_monto = IQR(rfm_df['Monto'])

  cat_rec = categorizacion(rfm_df['Recencia'], lim_rec, reciencia= True)
  cat_freq = categorizacion(rfm_df['Frecuencia'], lim_freq, reciencia=False)
  cat_monto = categorizacion(rfm_df['Monto'], lim_monto, reciencia=False)

  # Campos de puntaje
  rfm_df['puntaje_recencia'] = cat_rec[0]
  rfm_df['puntaje_frecuencia'] = cat_freq[0]
  rfm_df['puntaje_monto'] = cat_monto[0]

  # Campos de rango 
  rfm_df['rango_recencia'] = cat_rec[1]
  rfm_df['rango_frecuencia'] = cat_freq[1]
  rfm_df['rango_monto'] = cat_monto[1]
  
  rfm_df['score'] = rfm_df['puntaje_recencia'].astype(str)\
     + rfm_df['puntaje_frecuencia'].astype(str)\
     + rfm_df['puntaje_monto'].astype(str)
  
  rfm_df['score'] = rfm_df['score'].astype(int)

  #rfm_df['Score'] = rfm_df['puntaje_recencia'] + rfm_df['puntaje_frecuencia'] + rfm_df['puntaje_monto']
  
  rfm_df['Valor'] = rfm_df['score'].apply(lambda x: case_when(x))

  return(rfm_df)


In [67]:
# Carga de la base
inpath = "C:/Users/Diego Torres/OneDrive/Datasets/Manar/"

tb_ini = pd.read_excel(inpath + 'RFM_pruebas.xlsx',
                       sheet_name="Hoja1")

In [68]:
#################
# Descriptivos
print(tb_ini.sample(5))
print(' ')
print('Summary:')
tb_ini.info()
print(' ')
print('Descriptivos:')
print(tb_ini.describe().round(2).transpose())
print(' ')
print('Nulos por campo:')
print(tb_ini.isnull().sum())  # total de nulls por variable
print(' ')
print('Dimensiones:')
print(tb_ini.shape)

        codigo cliente  nombre cliente  codigo producto  \
62961        112192191       112192191         32408002   
11592      13330007157     13330007157         31000249   
101905       113249816       113249816         31000686   
28801        113313446       113313446         31000705   
55864        112814214       112814214         32104001   

                                          nombre producto fecha venta  \
62961               DIF MT-IBC MENSUALIDAD CEN FINANCIERO  2022-05-27   
11592   DIF MANTENIMIENTO A DISTANCIA DE PROGRAMAS Y E...  2022-01-27   
101905  **EXC EB - MENSUALIDAD VARIABLE FACTURA ELECTR...  2022-08-22   
28801   MT-IBC MENSUALIDAD FIJA ADMINISTRACION CEN-TEN...  2022-03-02   
55864        MT-IBC MENSUALIDAD NUMERO TRANSACCIONES CENT  2022-05-03   

        cantidad facturado  valor facturado                 Tipo  
62961                    1          82934.0       CEN FINANCIERO  
11592                    1          58943.0    CEN TRANSACCIONAL  
10190

In [73]:
####   RFM CLIENTES

# esta linea genera la fecha a la que comparar
hoy = max(tb_ini['fecha venta'])

tb_grp = tb_ini\
    .groupby('codigo cliente')\
    .agg(
        ultimo_dia=('fecha venta', 'max'),
        Frecuencia=('cantidad facturado', 'sum'),
        Monto=('valor facturado', 'sum'),
        nombre_cliente=('nombre cliente', 'unique'))\
    .assign(Recencia=lambda x: (hoy - x['ultimo_dia']).dt.days)\
    .sort_values('codigo cliente')
    #.assign(Recencia=(tb['recencia'].apply(lambda y: y.days)))

print(tb_grp.sample(5))
print(' ')
print('Dimensiones:')
print(tb_grp.shape)
print(' ')
print('Summary:')
tb_grp.info()

               ultimo_dia  Frecuencia       Monto  nombre_cliente  Recencia
codigo cliente                                                             
113238937      2022-08-25          12  65028305.0     [113238937]         6
113189674      2022-08-25          24   1688471.0     [113189674]         6
133110697534   2022-08-25          40   2318120.0  [133110697534]         6
113350895      2022-08-02          14   2737785.0     [113350895]        29
113265056      2022-06-02          12    985461.0     [113265056]        90
 
Dimensiones:
(3822, 5)
 
Summary:
<class 'pandas.core.frame.DataFrame'>
Index: 3822 entries, 1141132 to 139110576375
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   ultimo_dia      3822 non-null   datetime64[ns]
 1   Frecuencia      3822 non-null   int64         
 2   Monto           3822 non-null   float64       
 3   nombre_cliente  3822 non-null   object        
 4   

In [76]:
#############################
### TABLA RESULTADO
base_resultado = puntajes_RFM(tb_grp)

print(base_resultado.sample(5))
print(' ')
print('Dimensiones:')
print(base_resultado.shape)
print(' ')
print('Summary:')
base_resultado.info()

               ultimo_dia  Frecuencia      Monto nombre_cliente  Recencia  \
codigo cliente                                                              
112852023      2022-08-25          27  2310646.0    [112852023]         6   
112972975      2022-08-25          34  3137253.0    [112972975]         6   
113313446      2022-08-02          16  2339799.0    [113313446]        29   
113394884      2022-08-02           3   270000.0    [113394884]        29   
113300856      2022-08-02          14   570746.0    [113300856]        29   

                puntaje_recencia  puntaje_frecuencia  puntaje_monto  \
codigo cliente                                                        
112852023                      5                   4              4   
112972975                      5                   4              4   
113313446                      5                   2              4   
113394884                      5                   1              1   
113300856                      5  